In [2]:
import pandas as pd
import numpy as np

In [3]:
def safe_convert(col):
    col_cleaned = col.replace(',', '', regex=True)
    return pd.to_numeric(col_cleaned, errors='coerce')

setpt = pd.read_csv('setpoint_publish(in).csv')
setpt = setpt.apply(lambda col: safe_convert(col) if col.name != 'var' else col)
setpt



,var,opt_mean,opt_sig,drift_mean,drift_sig,excursion_mean,excursion_sig
0,x1,22.1,0.295,22.111,0.442,22.115,0.5900
1,x2,44.8,1.045,46.010,1.840,46.547,2.3270
2,x4,0.4,0.023,0.368,0.027,0.304,0.0670
3,x5,150000.0,8000.000,135000.000,9000.000,120000.000,16000.0000
4,x7,1206.0,48.240,1266.300,75.978,1266.300,88.6410
5,x8,87.0,5.220,91.176,6.686,107.880,12.2264
6,x9,454.0,19.673,456.270,22.814,599.280,69.9160
7,x17,5492.0,256.293,5526.050,349.983,5552.410,425.6850


In [4]:
def generate_normal_data(mean, std_dev, size=5000, seed=None):
    return np.random.normal(loc=mean, scale=std_dev, size=size)

In [5]:
datasets = {}
for data_name in ["opt", "drift", "excursion"]:
    rows = []
    for var, mean, sig in zip(setpt['var'], setpt[f'{data_name}_mean'], setpt[f'{data_name}_sig']):
        rows.append(generate_normal_data(mean, sig, seed=42))
    
    datasets[data_name] = pd.DataFrame(np.column_stack(rows), columns=setpt['var'])

In [6]:
datasets['excursion']

var,x1,x2,x4,x5,x7,x8,x9,x17
0,22.296999,46.001922,0.365425,121787.139367,1277.172362,77.991149,560.783442,4883.038540
1,21.820172,49.842250,0.279921,89887.051724,1250.926336,123.242473,672.224444,5573.730389
2,22.142361,48.189130,0.412134,143487.892692,1242.310627,92.628487,655.428648,5133.593223
3,21.540828,47.276837,0.191748,140262.796946,1317.164984,95.102056,679.790155,5840.677722
4,21.136118,43.321086,0.284692,124894.643543,1393.231516,88.262338,636.584253,5384.610214
...,...,...,...,...,...,...,...,...
4995,21.898511,49.403612,0.314938,92135.198873,1264.445325,107.241009,569.202100,6644.117826
4996,21.484704,45.228665,0.324778,121666.259085,1198.514994,117.570794,534.988954,5326.581897
4997,22.596456,46.648719,0.216904,129850.270655,1367.045221,98.658477,630.840769,5440.129117
4998,22.126658,50.740278,0.383283,122187.544203,1335.887576,111.884807,572.820190,5333.980978
